In [1]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import re
from tqdm import tqdm
from typing import Optional, Any, List, Tuple, Dict

In [2]:
# Keywords request response
requests_df = pd.read_csv("keywords_req&resp_163519(22-28July).csv")
requests_df.shape

(930730, 6)

In [3]:
requests_df.head()

,trimmed_keyword,keywords,word_cnt,request,response,RR
0,lipstick,lipstick,1,213661,212057,99.249278
1,alpsgoodnessrosemarywaterrefillpack,alps goodness rosemary water refill pack,6,98715,24434,24.752064
2,sunscreen,sunscreen,1,86398,86398,100.000000
3,facewash,face wash,2,76316,76004,99.591174
4,foundation,foundation,1,61821,61821,100.000000


In [5]:
# Keyword campaign level data
campaign_keywords = pd.read_csv("targetted_keywords_in_campaign_29July_163519.csv")
campaign_keywords.shape

(67335, 11)

In [6]:
# Cleaning & filtering the Data

# Step 1: Filter the data where is_negative is 'False'
filtered_df = campaign_keywords[campaign_keywords['is_negative'] == False]

# Step 2: Filter the data where status_type is 'ACTIVE'
filtered_df = filtered_df[filtered_df['status_type'] == 'ACTIVE']

# Step 2: Group by 'marketing_campaign_id' and 'text'
grouped_df = filtered_df.groupby(['keyword_id','marketing_campaign_id', 'text', 'match_type'], as_index=False).first()

# Step 3: Drop duplicate combinations (this is automatically handled by the groupby and first combination)
cleaned_camp_key_df = grouped_df.drop_duplicates(subset=['keyword_id','marketing_campaign_id', 'text', 'match_type'])

In [7]:
cleaned_camp_key_df.head()

,keyword_id,marketing_campaign_id,text,match_type,marketplace_client_id,agency_id,is_negative,status_type,bidding_value,bidding_currency,bidding_value_usd
0,135196,375534,sheet mask,PHRASE,163519,122,False,ACTIVE,30.0,INR,0.4059
1,135197,375534,face mask,PHRASE,163519,122,False,ACTIVE,25.0,INR,0.3383
2,135198,375534,korean mask,PHRASE,163519,122,False,ACTIVE,25.0,INR,0.3383
3,135199,375534,mask,PHRASE,163519,122,False,ACTIVE,25.0,INR,0.3383
4,389279,551819,Pilgrim Serum,BROAD,163519,122,False,ACTIVE,30.0,INR,0.4059


In [8]:
cleaned_camp_key_df.shape

(44974, 11)

In [9]:
campaign_keywords[campaign_keywords['status_type'] == "ACTIVE"]

,marketplace_client_id,agency_id,keyword_id,marketing_campaign_id,text,is_negative,match_type,status_type,bidding_value,bidding_currency,bidding_value_usd
0,163519,122,1268001,245954,face wash,True,EXACT,ACTIVE,0.00,NaN,0.0000
1,163519,122,1268003,245954,vitamin c face wash,True,EXACT,ACTIVE,0.00,NaN,0.0000
2,163519,122,1268004,245954,face scrub,True,EXACT,ACTIVE,0.00,NaN,0.0000
3,163519,122,1268002,245954,face wash for oily skin,True,EXACT,ACTIVE,0.00,NaN,0.0000
17,163519,122,1265600,290230,best body wash for glowing skin,True,EXACT,ACTIVE,0.00,NaN,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
67330,163519,122,769142,668804,best under eye patches,False,EXACT,ACTIVE,16.67,INR,0.2256
67331,163519,122,769145,668804,eye patches for dark circles,False,EXACT,ACTIVE,16.67,INR,0.2256
67332,163519,122,756625,668804,dark circle remover under eye patch,False,EXACT,ACTIVE,16.67,INR,0.2256
67333,163519,122,769158,668804,cooling eye patches,False,EXACT,ACTIVE,16.67,INR,0.2256


### Subset the data for keyword & match type 

In [10]:
keyword_match_df = cleaned_camp_key_df[['text', 'match_type']]
keyword_match_df = keyword_match_df.drop_duplicates(subset=['text', 'match_type'])
keyword_match_df.shape

(31508, 2)

In [11]:
keyword_match_df.head()

,text,match_type
0,sheet mask,PHRASE
1,face mask,PHRASE
2,korean mask,PHRASE
3,mask,PHRASE
4,Pilgrim Serum,BROAD


In [12]:
len(keyword_match_df['text'].unique())

23865

In [13]:
# Filter out EXACT & Phrase match type keywords
exact_keywords_df = keyword_match_df[keyword_match_df['match_type']=="EXACT"]
phrase_keywords_df = keyword_match_df[keyword_match_df['match_type']!="EXACT"]
# broad_keywords_df = keyword_match_df[keyword_match_df['match_type']=="BROAD"]

In [14]:
phrase_keywords_df.shape  # broad 584, phrase 7, exact 413

(15424, 2)

### Exact Matching Handling

In [15]:
# For Exact we have to direct map the request volume
exact_keywords_req_df = pd.merge(exact_keywords_df, requests_df[['keywords', 'request', 'response', 'RR']], left_on='text', right_on='keywords', how='left')
print(exact_keywords_req_df.isnull().sum())
exact_keywords_req_df = exact_keywords_req_df.drop('keywords', axis=1)
exact_keywords_req_df = exact_keywords_req_df.fillna(0)

text             0
match_type       0
keywords      4816
request       4816
response      4816
RR            4816
dtype: int64


In [16]:
exact_keywords_req_df.head()

,text,match_type,request,response,RR
0,Minimalist,EXACT,0.0,0.0,0.0
1,Plum,EXACT,0.0,0.0,0.0
2,Serum,EXACT,0.0,0.0,0.0
3,Plum Serum,EXACT,0.0,0.0,0.0
4,henna powder,EXACT,1499.0,1499.0,100.0


In [17]:
exact_keywords_req_df.shape

(16084, 5)

### Phrase Matching Handling

In [18]:
# Handling the phrase variations by Extracting the phrase variation of phrase matched targetted keywords

# Function for the manipulations on the targeted keywords
def targeted_keywords_manipulation(targeted_keywords) -> pd.DataFrame:
    targeted_keywords["lower_tar_keyword"] = (targeted_keywords["text"].str.lower().str.strip())
    targeted_keywords = targeted_keywords.loc[(~(targeted_keywords["lower_tar_keyword"].isna())) & (targeted_keywords["lower_tar_keyword"] != "")]    
    return targeted_keywords.lower_tar_keyword

# Function for the manipulations on the search queries
def search_query_manipulation(search_queries)->Tuple[pd.DataFrame, List[str]]:
    search_queries["lower_search_keyword"] = search_queries["keywords"].str.lower().str.strip()
    search_query_df_v2 = (
        search_queries.groupby(["trimmed_keyword", "lower_search_keyword", "word_cnt"])
                                .agg({"request": "sum", "response": "sum"})
                                .reset_index()
                                .sort_values(by="request", ascending=False)
                                )

    phrase_df = search_query_df_v2.copy()
    phrase_df = phrase_df.rename(
                                columns={
                                    "lower_search_keyword": "phrase_keyword",
                                    "request": "phrase_request",
                                    "response": "phrase_response_cnt",
                                }
                                ).drop(columns=["trimmed_keyword", "word_cnt"])
    
    # Create a list with phrase candidates
    phrase_set_lst = phrase_df["phrase_keyword"].tolist()
    return phrase_df, phrase_set_lst

# Function to find the phrase variations
def find_phrase_variations(targetted_keywords: List[str], search_query: List[str], score_cutoff=100):
    targeted_query_len = len(targetted_keywords)
    
    filtered_phrase_set = [query for query in search_query if targetted_keywords in query]
    
    matches = process.extract(targetted_keywords, filtered_phrase_set, scorer=fuzz.token_set_ratio, limit=None)
    score_filtered_lst = [key for key, score, _ in matches if score == score_cutoff]
    
    return score_filtered_lst

In [19]:
cleaned_phrase_keywords = targeted_keywords_manipulation(phrase_keywords_df)
cleaned_phrase_keywords = pd.DataFrame(phrase_keywords_df) # dataset should be in dataframe
cleaned_phrase_keywords.head()

C:\Users\mayur\AppData\Local\Temp\ipykernel_12672\1549867186.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targeted_keywords["lower_tar_keyword"] = (targeted_keywords["text"].str.lower().str.strip())


,text,match_type,lower_tar_keyword
0,sheet mask,PHRASE,sheet mask
1,face mask,PHRASE,face mask
2,korean mask,PHRASE,korean mask
3,mask,PHRASE,mask
4,Pilgrim Serum,BROAD,pilgrim serum


In [20]:
cleaned_phrase_keywords.shape

(15424, 3)

In [21]:
phrase_df, phrase_set_lst = search_query_manipulation(requests_df)
phrase_set_lst   # This has to be in series

['lipstick',
 'alps goodness rosemary water refill pack',
 'sunscreen',
 'face wash',
 'foundation',
 'dermdoc face serum',
 'serum',
 'lip balm',
 'dermdoc underarm darkness',
 'alps goodness rose water',
 'shampoo',
 'toner',
 'rosemary',
 'hair serum',
 'rosemary water',
 'concealer',
 'face pack',
 'eyeliner',
 'body lotion',
 'body wash',
 'good vibes face wash',
 'eyeshadow',
 'maybelline lipstick',
 'primer',
 'ny bae pro strobe cream 3 in 1 foundation',
 'faces canada foundation',
 'cetaphil cleanser',
 'lip gloss',
 'mars lipstick',
 'nail polish',
 'alps goodness rosemary water',
 'hair mask',
 'moisturizer for face',
 'mama earth face wash',
 'kajal',
 'eye liner waterproof',
 'night cream',
 'vitamin c face serum',
 'compact',
 'skincare',
 'rose water',
 'dot and key sunscreen',
 'simple face wash',
 'loreal shampoo',
 'face moisturizer',
 'face serum',
 'dermdoc',
 'niacinamide serum',
 'conditioner',
 'mascara',
 'highlighter',
 'swiss beauty lipstick',
 'kajal waterproo

In [22]:
# Finding Sub-queries
tqdm.pandas(desc="Finding the subqueries")
cleaned_phrase_keywords.loc[:, "matching_targets"] = cleaned_phrase_keywords["lower_tar_keyword"].progress_apply(lambda x: find_phrase_variations(x, phrase_set_lst))
cleaned_phrase_keywords

Finding the subqueries: 100%|████████████████████████████████████████████████████| 15424/15424 [56:58<00:00,  4.51it/s]


,text,match_type,lower_tar_keyword,matching_targets
0,sheet mask,PHRASE,sheet mask,"[sheet mask, garnier sheet mask, good vibes sh..."
1,face mask,PHRASE,face mask,"[face mask, good vibes face mask, korean face ..."
2,korean mask,PHRASE,korean mask,"[korean mask, tomato korean mask, korean mask ..."
3,mask,PHRASE,mask,"[hair mask, sheet mask, face mask, golden peel..."
4,Pilgrim Serum,BROAD,pilgrim serum,"[pilgrim serum, pilgrim serum aha bha pha, pil..."
...,...,...,...,...
44851,Insight Foundation,PHRASE,insight foundation,"[insight foundation, insight foundation hd, mn..."
44852,Insight Cosmetics foundation,PHRASE,insight cosmetics foundation,"[insight cosmetics foundation, insight cosmeti..."
44853,Foundation,PHRASE,foundation,"[foundation, ny bae pro strobe cream 3 in 1 fo..."
44971,Mist,PHRASE,mist,"[plum body mist, body mist, face mist, aqualog..."


In [23]:
filtered_df = cleaned_phrase_keywords[cleaned_phrase_keywords["matching_targets"].apply(len) == 0]

phrase_matched_keywords = cleaned_phrase_keywords.explode("matching_targets")
print("Pre Shape: ", phrase_matched_keywords.shape)
phrase_matched_keywords = phrase_matched_keywords.drop_duplicates(subset=['lower_tar_keyword', 'matching_targets'])
print("Post Shape: ", phrase_matched_keywords.shape)

Pre Shape:  (3023339, 4)
Post Shape:  (1609990, 4)


In [24]:
filtered_df.shape

(2248, 4)

In [26]:
phrase_matched_keywords.head()

,text,match_type,lower_tar_keyword,matching_targets
0,sheet mask,PHRASE,sheet mask,sheet mask
0,sheet mask,PHRASE,sheet mask,garnier sheet mask
0,sheet mask,PHRASE,sheet mask,good vibes sheet mask
0,sheet mask,PHRASE,sheet mask,sheet mask combo
0,sheet mask,PHRASE,sheet mask,lakme face sheet mask


In [27]:
# Adding request volume to phrase_matched_keywords
phrase_matched_keywords = pd.merge(phrase_matched_keywords, phrase_df[["phrase_keyword", "phrase_request", "phrase_response_cnt"]], how="inner", left_on="matching_targets", right_on="phrase_keyword")
phrase_matched_keywords = phrase_matched_keywords.drop('phrase_keyword', axis=1)
phrase_matched_keywords['phrase_response_rate'] = phrase_matched_keywords['phrase_response_cnt']*100/ phrase_matched_keywords['phrase_request']
phrase_matched_keywords.head()

,text,match_type,lower_tar_keyword,matching_targets,phrase_request,phrase_response_cnt,phrase_response_rate
0,sheet mask,PHRASE,sheet mask,sheet mask,14900,14854,99.691275
1,mask,PHRASE,mask,sheet mask,14900,14854,99.691275
2,sheet mask,PHRASE,sheet mask,garnier sheet mask,1069,406,37.979420
3,mask,PHRASE,mask,garnier sheet mask,1069,406,37.979420
4,garnier sheet mask,BROAD,garnier sheet mask,garnier sheet mask,1069,406,37.979420


In [28]:
phrase_matched_keywords[phrase_matched_keywords['text']=='heineken']

,text,match_type,lower_tar_keyword,matching_targets,phrase_request,phrase_response_cnt,phrase_response_rate


In [29]:
phrase_matched_keywords[phrase_matched_keywords['text']=='heineken']['phrase_request'].sum()

0

In [53]:
phrase_matched_keywords.to_csv("phrase_testing.csv", index=False)

In [30]:
phrase_keywords_req_df = (phrase_matched_keywords.groupby(['text', 'match_type'])
                                .agg({"phrase_request": "sum", "phrase_response_cnt": "sum"})
                                .reset_index()
                                .sort_values(by="phrase_request", ascending=False))

In [31]:
phrase_keywords_req_df.head()

,text,match_type,phrase_request,phrase_response_cnt
3410,face,PHRASE,1166979,993072
6342,lipstick,BROAD,765158,694806
2303,cream,PHRASE,756595,538871
9385,serum,PHRASE,741087,666411
5110,hair,BROAD,637892,472139


In [32]:
phrase_keywords_req_df.shape

(10923, 4)

In [33]:
phrase_keywords_req_df[phrase_keywords_req_df['text']=='heineken']

,text,match_type,phrase_request,phrase_response_cnt


In [34]:
# Renaming the phrase keyword request and response columns
phrase_keywords_req_df = phrase_keywords_req_df.rename(columns={"phrase_request": "request", "phrase_response_cnt": "response"})
phrase_keywords_req_df['RR'] = phrase_keywords_req_df['response']*100/phrase_keywords_req_df['request']

In [35]:
phrase_keywords_req_df[phrase_keywords_req_df['text']=='sunlight']

,text,match_type,request,response,RR


### Combining all match type requests and responses in single dataframe

In [36]:
# combining all match types keywords
combined_keyword_match_df = pd.concat([exact_keywords_req_df, phrase_keywords_req_df], axis=0, ignore_index=True)
combined_keyword_match_df.shape

(27007, 5)

In [37]:
combined_keyword_match_df.head()

,text,match_type,request,response,RR
0,Minimalist,EXACT,0.0,0.0,0.0
1,Plum,EXACT,0.0,0.0,0.0
2,Serum,EXACT,0.0,0.0,0.0
3,Plum Serum,EXACT,0.0,0.0,0.0
4,henna powder,EXACT,1499.0,1499.0,100.0


In [38]:
combined_keyword_match_df.isnull().sum()

text          0
match_type    0
request       0
response      0
RR            0
dtype: int64

In [39]:
combined_keyword_match_df.dtypes

text           object
match_type     object
request       float64
response      float64
RR            float64
dtype: object

In [40]:
combined_keyword_match_df[combined_keyword_match_df['text']=='sunlight']

,text,match_type,request,response,RR


In [41]:
combined_keyword_match_df.head()

,text,match_type,request,response,RR
0,Minimalist,EXACT,0.0,0.0,0.0
1,Plum,EXACT,0.0,0.0,0.0
2,Serum,EXACT,0.0,0.0,0.0
3,Plum Serum,EXACT,0.0,0.0,0.0
4,henna powder,EXACT,1499.0,1499.0,100.0


In [42]:
cleaned_camp_key_df.head()

,keyword_id,marketing_campaign_id,text,match_type,marketplace_client_id,agency_id,is_negative,status_type,bidding_value,bidding_currency,bidding_value_usd
0,135196,375534,sheet mask,PHRASE,163519,122,False,ACTIVE,30.0,INR,0.4059
1,135197,375534,face mask,PHRASE,163519,122,False,ACTIVE,25.0,INR,0.3383
2,135198,375534,korean mask,PHRASE,163519,122,False,ACTIVE,25.0,INR,0.3383
3,135199,375534,mask,PHRASE,163519,122,False,ACTIVE,25.0,INR,0.3383
4,389279,551819,Pilgrim Serum,BROAD,163519,122,False,ACTIVE,30.0,INR,0.4059


### Merging campaigns and its targetted keywords with request tags based on Match types

In [43]:
tagged_camp_key_df = pd.merge(cleaned_camp_key_df, combined_keyword_match_df, on=['text', 'match_type'], how='left')
tagged_camp_key_df.shape

(44974, 14)

In [44]:
tagged_camp_key_df.isnull().sum()

keyword_id                  0
marketing_campaign_id       0
text                        0
match_type                  0
marketplace_client_id       0
agency_id                   0
is_negative                 0
status_type                 0
bidding_value               0
bidding_currency            0
bidding_value_usd           0
request                  5452
response                 5452
RR                       5452
dtype: int64

In [45]:
tagged_camp_key_df = tagged_camp_key_df.fillna('0')

In [46]:
tagged_camp_key_df.dtypes

keyword_id                 int64
marketing_campaign_id      int64
text                      object
match_type                object
marketplace_client_id      int64
agency_id                  int64
is_negative                 bool
status_type               object
bidding_value            float64
bidding_currency          object
bidding_value_usd        float64
request                   object
response                  object
RR                        object
dtype: object

In [47]:
# Function to determine the tag based on the number of requests
def get_reach_tag(requests):
    if requests < 10:
        return 'Very low Reach'
    elif 10 <= requests < 100:
        return 'Low Reach'
    elif 100 <= requests < 1000:
        return 'Moderate Reach'
    else:
        return 'High Reach'

tagged_camp_key_df[['request', 'response', 'RR']] = tagged_camp_key_df[['request', 'response', 'RR']].astype(int)
tagged_camp_key_df['reach_tag'] = tagged_camp_key_df['request'].apply(get_reach_tag)

In [48]:
tagged_camp_key_df.head()

,keyword_id,marketing_campaign_id,text,match_type,marketplace_client_id,agency_id,is_negative,status_type,bidding_value,bidding_currency,bidding_value_usd,request,response,RR,reach_tag
0,135196,375534,sheet mask,PHRASE,163519,122,False,ACTIVE,30.0,INR,0.4059,27083,22420,82,High Reach
1,135197,375534,face mask,PHRASE,163519,122,False,ACTIVE,25.0,INR,0.3383,43877,34802,79,High Reach
2,135198,375534,korean mask,PHRASE,163519,122,False,ACTIVE,25.0,INR,0.3383,45,22,48,Low Reach
3,135199,375534,mask,PHRASE,163519,122,False,ACTIVE,25.0,INR,0.3383,206881,149531,72,High Reach
4,389279,551819,Pilgrim Serum,BROAD,163519,122,False,ACTIVE,30.0,INR,0.4059,4154,3849,92,High Reach


In [49]:
tagged_camp_key_df[tagged_camp_key_df['text']=='heineken']

,keyword_id,marketing_campaign_id,text,match_type,marketplace_client_id,agency_id,is_negative,status_type,bidding_value,bidding_currency,bidding_value_usd,request,response,RR,reach_tag


In [50]:
shared_camp_key_df = tagged_camp_key_df[['agency_id', 'keyword_id', 'marketing_campaign_id', 'text', 'match_type', 'request', 'response', 'RR', 'reach_tag']]
shared_camp_key_df.shape

(44974, 9)

In [51]:
shared_camp_key_df[shared_camp_key_df['text']=='sunlight']

,agency_id,keyword_id,marketing_campaign_id,text,match_type,request,response,RR,reach_tag


In [52]:
shared_camp_key_df.to_csv("targeted_keyword_with_ReqTag_163519.csv", index=False)